In [2]:
import cv2
import numpy as np
import sounddevice as sd
import mediapipe as mp
import threading
import time
import pygame

**Import Library**
* **cv2** : Untuk proses video webcam dan manipulasi gambar.
* **numpy** : Untuk operasi numerik, terutama array.
* **sounddevice** : Untuk input audio real-time dari mikrofon.
* **mediapipe** : Untuk deteksi tangan dan segmentasi background.
* **threading** : Untuk menjalankan proses audio dan video secara paralel.
* **time** : Untuk menghitung waktu dan delay.
* **pygame** : Untuk memainkan suara (audio playback).

In [3]:

# === Load background images ===
# Ubah ukuran semua gambar background menjadi 640x480
IMG_SIZE = (640, 480)
img_idle = cv2.resize(cv2.imread('stadion.jpg'), IMG_SIZE)         # Gambar default ketika tidak ada input
img_cheer = cv2.resize(cv2.imread('teriak.jpg'), IMG_SIZE)         # Gambar saat pengguna bersuara keras
img_clap = cv2.resize(cv2.imread('tepuktangan.jpg'), IMG_SIZE)     # Gambar saat tangan terdeteksi

* **`IMG_SIZE = (640, 480)`**
  Menentukan ukuran semua gambar background menjadi lebar 640 piksel dan tinggi 480 piksel. Ini agar ukuran gambar konsisten dan sesuai dengan resolusi webcam.

* **`cv2.imread('nama_file')`**
  Membaca gambar dari file yang disimpan di folder kerja. Misalnya, `'stadion.jpg'` adalah gambar latar saat keadaan idle (tidak ada suara atau gesture).

* **`cv2.resize(gambar, IMG_SIZE)`**
  Mengubah ukuran gambar yang dibaca supaya sesuai ukuran `IMG_SIZE` (640x480). Ini penting agar gambar latar dan frame webcam bisa disamakan ukurannya saat digabung.

* **`img_idle`**
  Menyimpan gambar default, yaitu stadion saat tidak ada input suara atau gesture.

* **`img_cheer`**
  Menyimpan gambar latar untuk kondisi saat pengguna berteriak/suara keras terdeteksi.

* **`img_clap`**
  Menyimpan gambar latar untuk kondisi saat tangan terdeteksi (gesture tepuk tangan).

In [4]:

# === Shared state untuk komunikasi antar thread ===
status_lock = threading.Lock()     # Lock agar tidak terjadi konflik saat mengakses status bersama
audio_status = "idle"              # Status berdasarkan input suara (idle atau cheer)
gesture_status = "idle"            # Status berdasarkan gesture tangan (idle atau clap)
current_status = "idle"            # Status akhir yang menentukan background dan suara

* **`threading.Lock()`**
  Membuat sebuah *lock* (kunci) yang digunakan untuk mengontrol akses ke variabel bersama (shared state) antar *thread*. Ini mencegah kondisi balapan (*race condition*) ketika beberapa *thread* mencoba membaca atau menulis variabel yang sama secara bersamaan.

* **`audio_status`**
  Variabel string yang menyimpan status input suara saat ini. Nilainya bisa `"idle"` jika suara normal atau diam, dan `"cheer"` jika suara keras terdeteksi.

* **`gesture_status`**
  Variabel string yang menyimpan status hasil deteksi gesture tangan. Nilainya bisa `"idle"` jika tidak ada tangan yang terdeteksi, dan `"clap"` jika tangan (gesture tepuk tangan) terdeteksi.

* **`current_status`**
  Variabel string yang menyimpan status akhir yang akan menentukan background gambar dan suara yang dimainkan. Status ini merupakan hasil gabungan prioritas antara `audio_status` dan `gesture_status`.


In [5]:
# === Inisialisasi MediaPipe untuk segmentasi dan deteksi tangan ===
mp_selfie = mp.solutions.selfie_segmentation
segmentor = mp_selfie.SelfieSegmentation(model_selection=0)       # Model segmentasi untuk memisahkan orang dari latar belakang
mp_hands = mp.solutions.hands

# === Inisialisasi pygame untuk memainkan file audio ===
pygame.mixer.init()
sound_cheer = pygame.mixer.Sound('teriak.wav')                     # Suara sorakan
sound_clap = pygame.mixer.Sound('tepuktangan.wav')                # Suara tepuk tangan

* **`mp.solutions.selfie_segmentation`**
  Modul MediaPipe yang digunakan untuk segmentasi gambar, memisahkan objek manusia (foreground) dari latar belakang (background).

* **`segmentor = mp_selfie.SelfieSegmentation(model_selection=0)`**
  Membuat instance segmentor dengan model pilihan 0, yang digunakan untuk memproses frame video dan menghasilkan mask segmentasi untuk memisahkan orang dari latar belakang.

* **`mp.solutions.hands`**
  Modul MediaPipe untuk deteksi dan pelacakan tangan secara real-time.

* **`pygame.mixer.init()`**
  Menginisialisasi modul mixer dari pygame untuk mengatur pemutaran audio.

* **`pygame.mixer.Sound('teriak.wav')`**
  Memuat file audio `teriak.wav` yang berisi suara sorakan, untuk dimainkan saat input suara keras terdeteksi.

* **`pygame.mixer.Sound('tepuktangan.wav')`**
  Memuat file audio `tepuktangan.wav` yang berisi suara tepuk tangan, untuk dimainkan saat gesture tangan (tepuk tangan) terdeteksi.

In [6]:
# === Parameter untuk debounce suara ===
last_active_time = 0              # Waktu terakhir suara keras terdeteksi
debounce_time = 1.0               # Delay agar suara tidak langsung dianggap hilang

clap_sound_playing = False        # Status apakah suara tepuk tangan sedang dimainkan

# === Parameter untuk debounce deteksi tangan ===
last_hand_seen_time = 0
hand_timeout = 0.5                # Jika tidak ada tangan selama 0.5 detik, status gesture kembali ke idle

* **`last_active_time`**
  Variabel ini menyimpan waktu (timestamp) terakhir kali suara keras terdeteksi. Ini digunakan untuk menentukan apakah suara masih aktif atau sudah hilang berdasarkan waktu.

* **`debounce_time`**
  Waktu delay (dalam detik) yang digunakan untuk *debounce* suara. Artinya, setelah suara keras berhenti terdeteksi, program akan menunggu selama `debounce_time` detik sebelum menganggap suara benar-benar hilang atau idle. Ini mencegah perubahan status yang terlalu cepat dan berulang.

* **`clap_sound_playing`**
  Boolean yang menunjukkan apakah suara tepuk tangan (`clap`) sedang diputar atau tidak. Ini mencegah pemutaran suara tepuk tangan yang berulang-ulang secara bersamaan.

* **`last_hand_seen_time`**
  Waktu (timestamp) terakhir kali tangan terdeteksi oleh kamera. Berguna untuk mengatur kapan gesture tangan dianggap tidak lagi aktif.

* **`hand_timeout`**
  Durasi waktu (dalam detik) yang digunakan untuk *debounce* gesture tangan. Jika selama `hand_timeout` detik tidak terdeteksi tangan, status gesture akan di-reset kembali menjadi "idle".

In [7]:
# === Fungsi untuk memperbarui status berdasarkan input suara ===
def update_status_audio(new_status):
    global audio_status
    with status_lock:
        if audio_status != new_status:
            print(f"[Audio Status] {audio_status} -> {new_status}")
            if new_status == "cheer":
                sound_clap.stop()                                # Hentikan tepuk tangan jika sedang main
                if audio_status != "cheer":
                    sound_cheer.play(loops=-1)                   # Mainkan suara teriak secara looping
            elif new_status == "idle":
                sound_cheer.stop()                               # Hentikan suara teriak
            audio_status = new_status
        else:
            # Pastikan suara tetap menyala kalau sudah cheer tapi tiba-tiba mati
            if new_status == "cheer" and not pygame.mixer.get_busy():
                sound_cheer.play(loops=-1)


**Penjelasan fungsi `update_status_audio(new_status)`:**
* **Fungsi ini bertugas memperbarui status suara (`audio_status`) secara thread-safe menggunakan `status_lock`.**
* **Parameter `new_status`** adalah status suara terbaru yang terdeteksi, bisa `"cheer"` (suara keras) atau `"idle"` (suara normal).
* Jika status berubah dari sebelumnya (`audio_status != new_status`):

  * Jika status baru `"cheer"`, maka suara tepuk tangan dihentikan supaya tidak tumpang tindih, lalu suara teriak (`sound_cheer`) diputar berulang.
  * Jika status baru `"idle"`, suara teriak dihentikan.
  * Status `audio_status` diperbarui ke status baru.
* Jika status tidak berubah tapi statusnya `"cheer"` dan suara tiba-tiba tidak sedang dimainkan (misal karena error), suara teriak akan dimainkan ulang untuk menjaga konsistensi.

Fungsi ini penting agar suara yang dimainkan sesuai dengan input suara dari pengguna dan memastikan tidak terjadi tumpang tindih suara atau suara berhenti tiba-tiba.


In [8]:

# === Fungsi untuk memperbarui status berdasarkan gesture tangan ===
def update_status_gesture(new_status):
    global gesture_status, clap_sound_playing
    with status_lock:
        if gesture_status != new_status:
            print(f"[Gesture Status] {gesture_status} -> {new_status}")
            gesture_status = new_status

            if new_status == "clap":
                if not clap_sound_playing:
                    print("Play sound clap")
                    sound_cheer.stop()                           # Hentikan suara cheer jika sedang main
                    sound_clap.play(loops=-1)                    # Mainkan suara tepuk tangan
                    clap_sound_playing = True
                    time.sleep(0.1)                              # Delay untuk stabilitas
            else:
                if clap_sound_playing:
                    print("Stop sound clap")
                    sound_clap.stop()
                    clap_sound_playing = False
                    time.sleep(0.05)

**Penjelasan fungsi `update_status_gesture(new_status)`:**

* Fungsi ini bertugas memperbarui status gesture tangan (`gesture_status`) secara thread-safe menggunakan `status_lock`.
* Parameter `new_status` adalah status gesture terbaru, bisa `"clap"` (deteksi tepuk tangan) atau `"idle"` (tidak ada gesture tepuk tangan).
* Jika status gesture berubah:

  * Jika status baru `"clap"` dan suara tepuk tangan belum dimainkan:

    * Suara cheer yang sedang diputar dihentikan supaya tidak tumpang tindih.
    * Suara tepuk tangan (`sound_clap`) dimainkan berulang.
    * Variabel `clap_sound_playing` di-set `True` sebagai penanda.
    * Diberikan delay kecil supaya suara bisa stabil dimainkan tanpa glitch.
  * Jika status baru bukan `"clap"` dan suara tepuk tangan sedang dimainkan:

    * Suara tepuk tangan dihentikan.
    * `clap_sound_playing` di-set `False`.
    * Delay kecil juga diberikan untuk stabilitas.

In [9]:
# === Fungsi untuk menentukan status akhir berdasarkan input audio dan gesture ===
def compute_final_status():
    global current_status
    with status_lock:
        prev_status = current_status
        if gesture_status == "clap":
            current_status = "clap"                              # Gesture lebih prioritas
        elif audio_status == "cheer":
            current_status = "cheer"
        else:
            current_status = "idle"
        return prev_status != current_status                     # Return apakah status berubah


**Penjelasan fungsi `compute_final_status()`:**

* Fungsi ini menentukan **status akhir** (`current_status`) yang akan dipakai untuk mengubah tampilan gambar dan suara berdasarkan dua input utama:

  * Status gesture tangan (`gesture_status`)
  * Status suara/audio (`audio_status`)

* Menggunakan `status_lock` untuk mencegah konflik data saat banyak thread mengakses status.

* **Prioritas status:**

  1. Jika gesture tangan mendeteksi tepuk tangan (`"clap"`), maka status akhir menjadi `"clap"`.
  2. Jika tidak ada gesture tepuk tangan, tapi suara keras terdeteksi (`"cheer"`), maka status akhir `"cheer"`.
  3. Jika tidak ada gesture atau suara keras, status menjadi `"idle"`.

* Fungsi mengembalikan nilai `True` jika status akhir berubah dibanding sebelumnya, sehingga program bisa tahu kapan harus mengupdate tampilan dan suara sesuai perubahan status.

Fungsi ini sangat penting untuk mengintegrasikan dua sumber input (gesture dan suara) dan menentukan apa yang sebenarnya harus ditampilkan atau dimainkan.


In [10]:

# === Fungsi callback untuk input suara real-time ===
def audio_callback(indata, frames, time_, status):
    global last_active_time
    if status:
        print(f"Audio status: {status}", flush=True)

    # Hitung volume suara dalam dB menggunakan RMS
    rms = np.sqrt(np.mean(indata**2))
    db = 20 * np.log10(rms + 1e-10)  # Tambahkan epsilon agar tidak log(0)
    now = time.time()
    print(f"Audio RMS dB: {db:.2f}", flush=True)

    threshold_db = -20              # Ambang batas suara
    if db > threshold_db:
        update_status_audio("cheer")
        last_active_time = now
    else:
        if now - last_active_time > debounce_time:
            update_status_audio("idle")

# === Thread untuk menangkap input suara terus-menerus ===
def audio_thread(device_index=None):
    kwargs = dict(channels=1, callback=audio_callback, samplerate=44100, blocksize=1024)
    if device_index is not None:
        kwargs['device'] = device_index
    try:
        with sd.InputStream(**kwargs):
            while True:
                time.sleep(0.1)
    except Exception as e:
        print(f"Audio stream error: {e}")


* **`audio_callback`**:

  * Fungsi ini dipanggil secara otomatis oleh `sounddevice` setiap kali ada data suara baru yang masuk.
  * Data suara dalam `indata` dihitung volumenya dalam desibel (dB).
  * Jika suara di atas threshold (-20 dB), maka status audio diupdate menjadi `"cheer"`.
  * Jika suara di bawah threshold dan sudah melewati waktu debounce, status diupdate ke `"idle"`.
  * `last_active_time` menyimpan kapan terakhir suara keras terdengar untuk menghindari perubahan status terlalu cepat (debounce).

* **`audio_thread`**:

  * Membuka input audio secara terus-menerus menggunakan `sounddevice.InputStream`.
  * Memanggil `audio_callback` setiap data audio baru masuk.
  * Dijalanin dalam thread terpisah agar tidak mengganggu proses utama program.



In [11]:
# === Fungsi untuk mengganti background menggunakan segmentasi MediaPipe ===
def apply_virtual_background(frame, bg_image):
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = segmentor.process(rgb)

    if result.segmentation_mask is None:
        return frame

    mask = result.segmentation_mask
    mask_fg = mask > 0.5                                             # Deteksi area manusia

    bg_resized = cv2.resize(bg_image, (frame.shape[1], frame.shape[0]))
    mask_fg_3c = np.repeat(mask_fg[:, :, np.newaxis], 3, axis=2)    # Buat mask jadi 3 channel

    fg = frame * mask_fg_3c                                          # Ambil bagian manusia dari frame
    bg = bg_resized * (~mask_fg_3c)                                  # Ambil bagian background dari gambar

    output = fg + bg                                                 # Gabungkan keduanya
    return output.astype(np.uint8)

**Penjelasan fungsi `apply_virtual_background`:**

* Fungsi ini mengambil frame video (dari kamera) dan sebuah gambar latar belakang (`bg_image`).
* Menggunakan MediaPipe Selfie Segmentation untuk membuat mask yang memisahkan manusia (foreground) dari latar belakang.
* Mask ini kemudian digunakan untuk menggabungkan manusia asli di depan dengan gambar latar belakang baru.
* Hasilnya adalah efek background virtual yang mengganti latar belakang asli dengan gambar lain.
* Fungsi ini mengembalikan frame baru yang sudah digabungkan, siap untuk ditampilkan.

In [12]:
#=== Fungsi utama loop tampilan webcam dan gesture ===
def display_loop():
    global current_status, last_hand_seen_time

    hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, IMG_SIZE[0])
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, IMG_SIZE[1])

    if not cap.isOpened():
        print("❌ Kamera tidak tersedia.")
        return

    cv2.namedWindow("Filter Stadion", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Filter Stadion", *IMG_SIZE)

    last_hand_seen_time = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, IMG_SIZE)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb)

        now = time.time()

        # Jika tangan terdeteksi
        if results.multi_hand_landmarks:
            tangan_terdeteksi = len(results.multi_hand_landmarks)
            last_hand_seen_time = now
            update_status_gesture("clap")
        else:
            # Jika tidak terlihat tangan terlalu lama, reset gesture
            if now - last_hand_seen_time > hand_timeout:
                update_status_gesture("idle")
            else:
                update_status_gesture("clap")

        _ = compute_final_status()

        # Tentukan background berdasarkan status akhir
        with status_lock:
            if current_status == "cheer":
                bg = img_cheer
            elif current_status == "clap":
                bg = img_clap
            else:
                bg = img_idle

        # Terapkan virtual background
        frame_out = apply_virtual_background(frame, bg)
        cv2.imshow("Filter Stadion", frame_out)

        # Tekan 'q' untuk keluar
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


**Penjelasan Fungsi `display_loop()`:**

* **Inisialisasi kamera dan MediaPipe Hands** untuk mendeteksi tangan secara real-time.
* **Membaca frame kamera**, lalu mendeteksi apakah ada tangan yang terlihat.
* Jika tangan terdeteksi, **update status gesture menjadi "clap"**.
* Jika tangan hilang lebih dari waktu tertentu (`hand_timeout`), status gesture kembali ke "idle".
* Menggunakan fungsi `compute_final_status()` untuk menentukan status akhir (prioritas gesture "clap" > audio "cheer" > idle).
* Berdasarkan status akhir, **menentukan gambar latar belakang yang digunakan**.
* **Mengaplikasikan virtual background** pada frame menggunakan segmentasi MediaPipe agar hanya latar belakang yang diganti, sementara orang tetap terlihat.
* Menampilkan hasil ke jendela OpenCV bernama "Filter Stadion".
* Menunggu input keyboard, jika `q` ditekan, program keluar dan membersihkan sumber daya.

Fungsi ini adalah **loop utama program** yang menjalankan webcam, mendeteksi gesture tangan, dan mengatur tampilan visual serta audio sesuai dengan input yang dideteksi, sehingga menciptakan pengalaman interaktif yang dinamis.


In [ ]:

# === Program utama ===
if __name__ == "__main__":
    print("Daftar device audio yang tersedia:")
    print(sd.query_devices())                                       # Menampilkan daftar perangkat audio

    device_index = None                                             # Ganti jika perlu memilih mikrofon spesifik

    # Mulai thread audio dan tampilan webcam
    threading.Thread(target=audio_thread, args=(device_index,), daemon=True).start()
    display_loop()

Daftar device audio yang tersedia:
   0 Microsoft Sound Mapper - Input, MME (2 in, 0 out)
>  1 Microphone Array (Realtek(R) Au, MME (2 in, 0 out)
   2 Microsoft Sound Mapper - Output, MME (0 in, 2 out)
<  3 Speakers (Realtek(R) Audio), MME (0 in, 2 out)
   4 Primary Sound Capture Driver, Windows DirectSound (2 in, 0 out)
   5 Microphone Array (Realtek(R) Audio), Windows DirectSound (2 in, 0 out)
   6 Primary Sound Driver, Windows DirectSound (0 in, 2 out)
   7 Speakers (Realtek(R) Audio), Windows DirectSound (0 in, 2 out)
   8 Speakers (Realtek(R) Audio), Windows WASAPI (0 in, 2 out)
   9 Microphone Array (Realtek(R) Audio), Windows WASAPI (2 in, 0 out)
  10 Speakers 1 (Realtek HD Audio output with SST), Windows WDM-KS (0 in, 2 out)
  11 Speakers 2 (Realtek HD Audio output with SST), Windows WDM-KS (0 in, 2 out)
  12 PC Speaker (Realtek HD Audio output with SST), Windows WDM-KS (2 in, 0 out)
  13 Stereo Mix (Realtek HD Audio Stereo input), Windows WDM-KS (2 in, 0 out)
  14 Microphone A

Audio RMS dB: -96.91
Audio RMS dB: -96.54
Audio RMS dB: -96.36
Audio RMS dB: -97.03
Audio RMS dB: -96.91
Audio RMS dB: -96.54
Audio RMS dB: -96.36
Audio RMS dB: -97.03
Audio RMS dB: -96.91
Audio RMS dB: -96.54
Audio RMS dB: -96.36
Audio RMS dB: -97.03
Audio RMS dB: -96.91
Audio RMS dB: -96.54
Audio RMS dB: -96.36
Audio RMS dB: -97.03
Audio RMS dB: -96.91
Audio RMS dB: -96.54
Audio RMS dB: -96.36
Audio RMS dB: -97.03
Audio RMS dB: -96.91
Audio RMS dB: -96.54
Audio RMS dB: -96.36
Audio RMS dB: -97.03
Audio RMS dB: -96.91
Audio RMS dB: -96.54
Audio RMS dB: -96.36
Audio RMS dB: -97.03
Audio RMS dB: -96.91
Audio RMS dB: -96.54
Audio RMS dB: -96.36
Audio RMS dB: -97.03
Audio RMS dB: -96.91
Audio RMS dB: -96.54
Audio RMS dB: -96.36
Audio RMS dB: -97.03
Audio RMS dB: -96.91
Audio RMS dB: -96.54
Audio RMS dB: -96.36
Audio RMS dB: -97.03
Audio RMS dB: -96.91
Audio RMS dB: -96.54
Audio RMS dB: -96.36
Audio RMS dB: -97.03
Audio RMS dB: -96.91
Audio RMS dB: -96.54
Audio RMS dB: -96.36
Audio RMS dB:

* **`if __name__ == "__main__":`**
  Menandai bahwa kode di bawahnya hanya dijalankan saat file ini dieksekusi langsung, bukan diimpor sebagai modul.

* **`sd.query_devices()`**
  Memanggil fungsi dari `sounddevice` untuk menampilkan semua perangkat audio (mikrofon dan speaker) yang tersedia di komputer. Berguna untuk memilih perangkat input suara yang diinginkan.

* **`device_index = None`**
  Variabel untuk menentukan indeks mikrofon yang dipakai. Jika `None`, akan otomatis pakai perangkat default.

* **`threading.Thread(...).start()`**
  Membuat dan menjalankan *thread* baru untuk fungsi `audio_thread` yang memantau suara secara real-time tanpa mengganggu jalannya program utama.

* **`display_loop()`**
  Memanggil fungsi utama yang menjalankan webcam, mendeteksi gesture, dan menampilkan output visual sesuai status yang diperbarui.

Secara keseluruhan, kode ini memulai program dengan:

1. Menampilkan daftar perangkat audio untuk referensi.
2. Memulai proses deteksi suara di background secara paralel.
3. Menampilkan webcam dengan filter interaktif berbasis suara dan gesture.

Jadi, input suara dan gesture bisa diproses bersamaan secara real-time.
